# Focused Learning: Iterative Code Re-generation with Agent-Based Approach

## Mục tiêu học tập

Notebook này tập trung vào khái niệm **Iterative Code Re-generation** (Section 3.2) - cốt lõi của CoderGen framework. Chúng ta sẽ:

1. Hiểu cách agent-based system hoạt động trong code generation
2. Implement iterative refinement loop với feedback
3. Xây dựng state management cho multi-iteration process
4. Tạo complete agent workflow từ generation đến validation

## Trích xuất từ paper

**Section 3.2: Iterative Code Re-generation** (trang 6)
- "The erroneous code snippet, along with the suggestions and the original instruction, are fed back into the language model"
- "This iterative cycle ensures that the generated code not only resolves the immediate issues but also improves in quality and robustness with each iteration"
- "The framework's ability to learn from its mistakes and adapt its code generation strategy based on real-time feedback"
- Figure 2 shows complete CoderGen architecture với feedback loops

## 1. Lý thuyết: Agent-Based Iterative Refinement

### 1.1. Agent Architecture cho Code Generation

Agent-based approach trong CoderGen có các components:

1. **State**: Current code, error history, iteration count
2. **Actions**: Generate, Execute, Analyze, Regenerate
3. **Transitions**: Based on execution results
4. **Memory**: Track improvements across iterations

### 1.2. Iterative Improvement Formula

$$Code_{n+1} = f(Code_n, Error_n, Feedback_n, Context)$$

Trong đó:
- $Code_n$: Code ở iteration thứ n
- $Error_n$: Error analysis từ execution
- $Feedback_n$: Suggestions và fixes
- $Context$: Task description và history

### 1.3. Convergence Strategy

Agent cần balance giữa:
- **Exploration**: Try different approaches
- **Exploitation**: Refine current approach
- **Termination**: Know when to stop

In [ ]:
import asyncio
import time
from typing import Dict, List, Optional, Tuple, Any, Callable
from dataclasses import dataclass, field
from enum import Enum, auto
import json
import subprocess
import tempfile
import os
from collections import deque
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from datetime import datetime

## 2. Agent State Definition

In [ ]:
class AgentAction(Enum):
    """Possible actions for the code generation agent"""
    GENERATE = auto()
    EXECUTE = auto()
    ANALYZE = auto()
    REGENERATE = auto()
    TERMINATE = auto()

class ExecutionStatus(Enum):
    """Status of code execution"""
    SUCCESS = "success"
    SYNTAX_ERROR = "syntax_error"
    RUNTIME_ERROR = "runtime_error"
    PARTIAL_SUCCESS = "partial_success"
    TIMEOUT = "timeout"

@dataclass
class AgentState:
    """Complete state of the code generation agent"""
    # Task information
    task_description: str
    task_requirements: List[str]
    
    # Current code state
    current_code: str = ""
    iteration: int = 0
    
    # Execution results
    last_execution_status: Optional[ExecutionStatus] = None
    last_error: Optional[str] = None
    test_results: Dict[str, bool] = field(default_factory=dict)
    
    # History tracking
    code_history: List[str] = field(default_factory=list)
    error_history: List[str] = field(default_factory=list)
    improvement_metrics: List[float] = field(default_factory=list)
    
    # Agent memory
    successful_patterns: List[str] = field(default_factory=list)
    failed_approaches: List[str] = field(default_factory=list)
    
    # Control parameters
    max_iterations: int = 5
    confidence_threshold: float = 0.8
    current_confidence: float = 0.0
    
    def update_iteration(self):
        """Update iteration and save current state to history"""
        self.iteration += 1
        if self.current_code:
            self.code_history.append(self.current_code)
        if self.last_error:
            self.error_history.append(self.last_error)
    
    def calculate_improvement(self) -> float:
        """Calculate improvement score based on test results"""
        if not self.test_results:
            return 0.0
        
        passed = sum(1 for result in self.test_results.values() if result)
        total = len(self.test_results)
        
        return passed / total if total > 0 else 0.0
    
    def should_terminate(self) -> bool:
        """Decide if agent should stop iterating"""
        # Success condition
        if self.last_execution_status == ExecutionStatus.SUCCESS:
            return True
        
        # Max iterations reached
        if self.iteration >= self.max_iterations:
            return True
        
        # High confidence achieved
        if self.current_confidence >= self.confidence_threshold:
            return True
        
        # No improvement in last 3 iterations
        if len(self.improvement_metrics) >= 3:
            recent_improvements = self.improvement_metrics[-3:]
            if all(imp == recent_improvements[0] for imp in recent_improvements):
                return True
        
        return False

## 3. Code Execution Environment

In [ ]:
class CodeExecutor:
    """Safe code execution environment with timeout and error capture"""
    
    def __init__(self, timeout: int = 30):
        self.timeout = timeout
        self.execution_count = 0
    
    def execute_code(self, code: str, test_cases: List[Dict[str, Any]]) -> Tuple[ExecutionStatus, Dict[str, Any]]:
        """Execute code with test cases and return results"""
        self.execution_count += 1
        
        # Create temporary file
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
            f.write(code)
            temp_file = f.name
        
        try:
            # First check syntax
            compile(code, temp_file, 'exec')
        except SyntaxError as e:
            os.unlink(temp_file)
            return ExecutionStatus.SYNTAX_ERROR, {
                "error": str(e),
                "line": e.lineno,
                "offset": e.offset
            }
        
        # Execute with timeout (simulated)
        results = self._simulate_execution(code, test_cases)
        
        os.unlink(temp_file)
        return results
    
    def _simulate_execution(self, code: str, test_cases: List[Dict[str, Any]]) -> Tuple[ExecutionStatus, Dict[str, Any]]:
        """Simulate code execution with test cases"""
        import random
        
        # Simulate different execution scenarios
        if "import" not in code:
            return ExecutionStatus.RUNTIME_ERROR, {
                "error": "ImportError: No imports found",
                "test_results": {f"test_{i}": False for i in range(len(test_cases))}
            }
        
        if "def" not in code:
            return ExecutionStatus.RUNTIME_ERROR, {
                "error": "No function definition found",
                "test_results": {f"test_{i}": False for i in range(len(test_cases))}
            }
        
        # Simulate test execution
        test_results = {}
        passed_count = 0
        
        for i, test in enumerate(test_cases):
            # Improve success rate with iterations
            success_probability = min(0.3 + (self.execution_count * 0.15), 0.9)
            test_passed = random.random() < success_probability
            test_results[f"test_{i}"] = test_passed
            if test_passed:
                passed_count += 1
        
        # Determine overall status
        if passed_count == len(test_cases):
            status = ExecutionStatus.SUCCESS
        elif passed_count > 0:
            status = ExecutionStatus.PARTIAL_SUCCESS
        else:
            status = ExecutionStatus.RUNTIME_ERROR
        
        return status, {
            "test_results": test_results,
            "passed": passed_count,
            "total": len(test_cases),
            "execution_time": random.uniform(0.1, 2.0)
        }

## 4. Feedback Generator

In [ ]:
class FeedbackGenerator:
    """Generate actionable feedback from execution results"""
    
    def __init__(self):
        self.feedback_templates = self._init_templates()
    
    def generate_feedback(self, state: AgentState, execution_result: Dict[str, Any]) -> Dict[str, Any]:
        """Generate structured feedback for code improvement"""
        feedback = {
            "iteration": state.iteration,
            "status": state.last_execution_status.value if state.last_execution_status else "unknown",
            "improvements": [],
            "specific_fixes": [],
            "confidence_boost": 0.0
        }
        
        # Analyze based on execution status
        if state.last_execution_status == ExecutionStatus.SYNTAX_ERROR:
            feedback["improvements"] = self._syntax_error_feedback(state.last_error)
            feedback["confidence_boost"] = -0.2
        
        elif state.last_execution_status == ExecutionStatus.RUNTIME_ERROR:
            feedback["improvements"] = self._runtime_error_feedback(state.last_error)
            feedback["confidence_boost"] = -0.1
        
        elif state.last_execution_status == ExecutionStatus.PARTIAL_SUCCESS:
            feedback["improvements"] = self._partial_success_feedback(execution_result)
            feedback["confidence_boost"] = 0.2
        
        # Add specific fixes based on patterns
        feedback["specific_fixes"] = self._generate_specific_fixes(state)
        
        # Calculate improvement direction
        feedback["improvement_direction"] = self._calculate_improvement_direction(state)
        
        return feedback
    
    def _init_templates(self) -> Dict[str, List[str]]:
        """Initialize feedback templates"""
        return {
            "syntax": [
                "Check indentation and brackets",
                "Ensure all strings are properly closed",
                "Verify function definitions have colons"
            ],
            "import": [
                "Add missing import statements",
                "Use 'from module import function' syntax",
                "Check module names are correct"
            ],
            "logic": [
                "Review the algorithm logic",
                "Add input validation",
                "Handle edge cases"
            ]
        }
    
    def _syntax_error_feedback(self, error: str) -> List[str]:
        """Generate feedback for syntax errors"""
        feedback = ["Fix syntax error: " + error]
        
        if "indent" in error.lower():
            feedback.append("Check indentation - use 4 spaces consistently")
        if "invalid syntax" in error.lower():
            feedback.append("Check for missing colons, brackets, or quotes")
        
        return feedback
    
    def _runtime_error_feedback(self, error: str) -> List[str]:
        """Generate feedback for runtime errors"""
        feedback = []
        
        if "ImportError" in error:
            feedback.append("Add missing import statements at the beginning")
            feedback.append("Check if the module name is correct")
        elif "NameError" in error:
            feedback.append("Define all variables and functions before use")
            feedback.append("Check for typos in variable names")
        elif "TypeError" in error:
            feedback.append("Check data types match function expectations")
            feedback.append("Add type conversion if needed")
        
        return feedback
    
    def _partial_success_feedback(self, results: Dict[str, Any]) -> List[str]:
        """Generate feedback for partial success"""
        feedback = []
        
        test_results = results.get("test_results", {})
        failed_tests = [k for k, v in test_results.items() if not v]
        
        if failed_tests:
            feedback.append(f"Failed tests: {', '.join(failed_tests)}")
            feedback.append("Review logic for edge cases")
            feedback.append("Add more comprehensive error handling")
        
        return feedback
    
    def _generate_specific_fixes(self, state: AgentState) -> List[Dict[str, str]]:
        """Generate specific code fixes based on state"""
        fixes = []
        
        # Check for common patterns in code
        if state.current_code:
            if "transformers" in state.task_description and "import transformers" not in state.current_code:
                fixes.append({
                    "issue": "Missing transformers import",
                    "fix": "Add 'from transformers import pipeline' at the top"
                })
            
            if "try:" not in state.current_code:
                fixes.append({
                    "issue": "No error handling",
                    "fix": "Wrap main logic in try-except block"
                })
        
        return fixes
    
    def _calculate_improvement_direction(self, state: AgentState) -> str:
        """Suggest improvement direction"""
        if state.iteration == 1:
            return "Focus on getting basic structure working"
        elif state.last_execution_status == ExecutionStatus.PARTIAL_SUCCESS:
            return "Refine edge case handling"
        elif len(state.error_history) > 2:
            return "Consider alternative approach"
        else:
            return "Continue iterative refinement"

## 5. Iterative Code Generator

In [ ]:
class IterativeCodeGenerator:
    """Generate and refine code based on feedback"""
    
    def __init__(self):
        self.generation_strategies = [
            self._basic_generation,
            self._error_aware_generation,
            self._pattern_based_generation
        ]
    
    def generate_initial_code(self, task_description: str, requirements: List[str]) -> str:
        """Generate initial code attempt"""
        # Simplified code generation
        code_template = '''
# Task: {task}
# Requirements: {requirements}

def solve_task(input_data):
    """Solve the given task"""
    # TODO: Implement solution
    result = process_input(input_data)
    return result

def process_input(data):
    """Process input data"""
    return data

# Test the solution
if __name__ == "__main__":
    test_input = "sample"
    print(solve_task(test_input))
'''
        
        return code_template.format(
            task=task_description,
            requirements=", ".join(requirements)
        )
    
    def regenerate_code(self, state: AgentState, feedback: Dict[str, Any]) -> str:
        """Regenerate code based on state and feedback"""
        # Choose strategy based on iteration
        strategy_index = min(state.iteration - 1, len(self.generation_strategies) - 1)
        strategy = self.generation_strategies[strategy_index]
        
        return strategy(state, feedback)
    
    def _basic_generation(self, state: AgentState, feedback: Dict[str, Any]) -> str:
        """Basic improvement strategy"""
        improved_code = state.current_code
        
        # Add imports if missing
        if "import" in str(feedback.get("improvements", [])):
            imports = self._generate_imports(state.task_description)
            improved_code = imports + "\n\n" + improved_code
        
        # Add error handling
        if "error handling" in str(feedback.get("improvements", [])):
            improved_code = self._add_error_handling(improved_code)
        
        return improved_code
    
    def _error_aware_generation(self, state: AgentState, feedback: Dict[str, Any]) -> str:
        """Generate code aware of previous errors"""
        # Analyze error patterns
        error_types = self._analyze_error_patterns(state.error_history)
        
        # Generate code avoiding known errors
        if "syntax" in error_types:
            return self._generate_syntax_safe_code(state)
        elif "import" in error_types:
            return self._generate_with_proper_imports(state)
        else:
            return self._generate_robust_code(state)
    
    def _pattern_based_generation(self, state: AgentState, feedback: Dict[str, Any]) -> str:
        """Use successful patterns from history"""
        # Find successful patterns
        if state.successful_patterns:
            base_pattern = state.successful_patterns[-1]
            return self._adapt_pattern(base_pattern, state.task_description)
        
        # Generate new approach
        return self._generate_alternative_approach(state)
    
    def _generate_imports(self, task_description: str) -> str:
        """Generate appropriate imports based on task"""
        imports = []
        
        if "classification" in task_description:
            imports.append("from transformers import pipeline")
        if "image" in task_description:
            imports.append("from PIL import Image")
            imports.append("import torchvision.transforms as transforms")
        if "pytorch" in task_description.lower():
            imports.append("import torch")
        
        return "\n".join(imports)
    
    def _add_error_handling(self, code: str) -> str:
        """Add error handling to code"""
        # Simple wrapper for demo
        wrapped = f'''try:
{chr(10).join("    " + line for line in code.split(chr(10)))}
except Exception as e:
    print(f"Error occurred: {{e}}")
    raise
'''
        return wrapped
    
    def _analyze_error_patterns(self, error_history: List[str]) -> List[str]:
        """Analyze patterns in error history"""
        patterns = []
        
        for error in error_history:
            if "syntax" in error.lower():
                patterns.append("syntax")
            elif "import" in error.lower():
                patterns.append("import")
            elif "type" in error.lower():
                patterns.append("type")
        
        return patterns
    
    def _generate_syntax_safe_code(self, state: AgentState) -> str:
        """Generate code with careful syntax"""
        template = '''# {task}

def main():
    """Main function with proper syntax"""
    try:
        # Implementation
        result = None
        return result
    except Exception as e:
        return f"Error: {{e}}"

if __name__ == "__main__":
    print(main())
'''
        return template.format(task=state.task_description)
    
    def _generate_with_proper_imports(self, state: AgentState) -> str:
        """Generate code with comprehensive imports"""
        imports = self._generate_imports(state.task_description)
        base_code = self._generate_syntax_safe_code(state)
        return imports + "\n\n" + base_code
    
    def _generate_robust_code(self, state: AgentState) -> str:
        """Generate robust code with validation"""
        return self._generate_with_proper_imports(state)
    
    def _adapt_pattern(self, pattern: str, task: str) -> str:
        """Adapt successful pattern to new task"""
        # Simple adaptation for demo
        return pattern.replace("# Task:", f"# Task: {task}")
    
    def _generate_alternative_approach(self, state: AgentState) -> str:
        """Generate completely different approach"""
        return self._generate_with_proper_imports(state)

## 6. Complete Iterative Agent

In [ ]:
class IterativeCodeAgent:
    """Complete agent implementing iterative code generation"""
    
    def __init__(self):
        self.executor = CodeExecutor()
        self.feedback_generator = FeedbackGenerator()
        self.code_generator = IterativeCodeGenerator()
        self.state_history = []
    
    async def generate_code(self, task_description: str, 
                          requirements: List[str], 
                          test_cases: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Main agent loop for code generation"""
        
        # Initialize state
        state = AgentState(
            task_description=task_description,
            task_requirements=requirements
        )
        
        # Generate initial code
        state.current_code = self.code_generator.generate_initial_code(
            task_description, requirements
        )
        
        # Main iteration loop
        while not state.should_terminate():
            state.update_iteration()
            print(f"\n=== Iteration {state.iteration} ===")
            
            # Execute code
            status, results = self.executor.execute_code(state.current_code, test_cases)
            state.last_execution_status = status
            state.test_results = results.get("test_results", {})
            
            if "error" in results:
                state.last_error = results["error"]
            
            # Calculate improvement
            improvement = state.calculate_improvement()
            state.improvement_metrics.append(improvement)
            
            print(f"Execution Status: {status.value}")
            print(f"Test Results: {results.get('passed', 0)}/{results.get('total', 0)} passed")
            print(f"Improvement Score: {improvement:.2f}")
            
            # Check if we should stop
            if state.should_terminate():
                break
            
            # Generate feedback
            feedback = self.feedback_generator.generate_feedback(state, results)
            state.current_confidence += feedback["confidence_boost"]
            
            print(f"Feedback: {feedback['improvements'][:2]}")
            
            # Regenerate code
            state.current_code = self.code_generator.regenerate_code(state, feedback)
            
            # Save state
            self.state_history.append(state.__dict__.copy())
            
            # Async delay to simulate processing
            await asyncio.sleep(0.1)
        
        # Final result
        return self._prepare_final_result(state)
    
    def _prepare_final_result(self, state: AgentState) -> Dict[str, Any]:
        """Prepare final result summary"""
        return {
            "success": state.last_execution_status == ExecutionStatus.SUCCESS,
            "final_code": state.current_code,
            "iterations": state.iteration,
            "improvement_trajectory": state.improvement_metrics,
            "final_test_results": state.test_results,
            "confidence": state.current_confidence,
            "error_count": len(state.error_history)
        }
    
    def visualize_iteration_process(self):
        """Visualize the iteration process"""
        if not self.state_history:
            print("No iteration history available")
            return
        
        iterations = [s["iteration"] for s in self.state_history]
        improvements = [s["improvement_metrics"][-1] if s["improvement_metrics"] else 0 
                       for s in self.state_history]
        confidence = [s["current_confidence"] for s in self.state_history]
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
        
        # Improvement over iterations
        ax1.plot(iterations, improvements, 'b-o', label='Test Success Rate')
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Success Rate')
        ax1.set_title('Code Improvement Over Iterations')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Confidence over iterations
        ax2.plot(iterations, confidence, 'g-s', label='Agent Confidence')
        ax2.axhline(y=0.8, color='r', linestyle='--', label='Confidence Threshold')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Confidence')
        ax2.set_title('Agent Confidence Evolution')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()

## 7. Agent Workflow Visualization

In [ ]:
def visualize_agent_workflow():
    """Create visual representation of agent workflow"""
    
    # Create directed graph
    G = nx.DiGraph()
    
    # Add nodes
    nodes = [
        ("START", {"color": "green", "size": 1000}),
        ("GENERATE", {"color": "lightblue", "size": 1200}),
        ("EXECUTE", {"color": "yellow", "size": 1200}),
        ("ANALYZE", {"color": "orange", "size": 1200}),
        ("FEEDBACK", {"color": "pink", "size": 1000}),
        ("REGENERATE", {"color": "lightcoral", "size": 1200}),
        ("SUCCESS", {"color": "lightgreen", "size": 1000}),
        ("TERMINATE", {"color": "red", "size": 1000})
    ]
    
    for node, attrs in nodes:
        G.add_node(node, **attrs)
    
    # Add edges
    edges = [
        ("START", "GENERATE", "Initial"),
        ("GENERATE", "EXECUTE", "Run"),
        ("EXECUTE", "ANALYZE", "Check"),
        ("ANALYZE", "SUCCESS", "All Pass"),
        ("ANALYZE", "FEEDBACK", "Errors"),
        ("FEEDBACK", "REGENERATE", "Improve"),
        ("REGENERATE", "EXECUTE", "Retry"),
        ("ANALYZE", "TERMINATE", "Max Iter"),
        ("SUCCESS", "TERMINATE", "Done")
    ]
    
    for src, dst, label in edges:
        G.add_edge(src, dst, label=label)
    
    # Layout
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # Draw
    plt.figure(figsize=(12, 8))
    
    # Draw nodes
    node_colors = [G.nodes[node]["color"] for node in G.nodes()]
    node_sizes = [G.nodes[node]["size"] for node in G.nodes()]
    
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                          node_size=node_sizes, alpha=0.9)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color='gray', 
                          arrows=True, arrowsize=20, width=2)
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
    
    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=8)
    
    plt.title("Iterative Code Generation Agent Workflow", fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize the workflow
visualize_agent_workflow()

## 8. Demo: Complete Agent Execution

In [ ]:
# Demo execution
async def demo_agent_execution():
    """Demonstrate the iterative agent in action"""
    
    # Initialize agent
    agent = IterativeCodeAgent()
    
    # Define task
    task = "Create a text classification function using transformers pipeline"
    requirements = [
        "Use HuggingFace transformers",
        "Handle empty input gracefully",
        "Return confidence scores"
    ]
    
    # Define test cases
    test_cases = [
        {"input": "This is great!", "expected": "positive"},
        {"input": "", "expected": "error"},
        {"input": "Terrible experience", "expected": "negative"}
    ]
    
    print("Starting Iterative Code Generation Agent")
    print("=" * 50)
    print(f"Task: {task}")
    print(f"Requirements: {requirements}")
    print(f"Test Cases: {len(test_cases)}")
    print("=" * 50)
    
    # Run agent
    result = await agent.generate_code(task, requirements, test_cases)
    
    # Display results
    print("\n" + "=" * 50)
    print("FINAL RESULTS")
    print("=" * 50)
    print(f"Success: {result['success']}")
    print(f"Total Iterations: {result['iterations']}")
    print(f"Final Confidence: {result['confidence']:.2f}")
    print(f"Error Count: {result['error_count']}")
    print(f"\nImprovement Trajectory: {[f'{x:.2f}' for x in result['improvement_trajectory']]}")
    
    print("\nFinal Code:")
    print("-" * 30)
    print(result['final_code'])
    
    # Visualize iteration process
    agent.visualize_iteration_process()
    
    return result

# Run the demo
await demo_agent_execution()

## 9. Analysis: Convergence Patterns

In [ ]:
# Analyze convergence patterns across multiple runs
async def analyze_convergence_patterns():
    """Run multiple experiments to analyze convergence"""
    
    tasks = [
        "Text classification with transformers",
        "Image classification with PyTorch",
        "Speech recognition pipeline",
        "Sentiment analysis function"
    ]
    
    results = []
    
    for task in tasks:
        agent = IterativeCodeAgent()
        result = await agent.generate_code(
            task, 
            ["Use appropriate library", "Add error handling"],
            [{"test": i} for i in range(3)]
        )
        results.append({
            "task": task,
            "iterations": result["iterations"],
            "success": result["success"],
            "final_score": result["improvement_trajectory"][-1] if result["improvement_trajectory"] else 0
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Iterations by task
    ax1 = axes[0, 0]
    ax1.bar(range(len(df)), df['iterations'])
    ax1.set_xticks(range(len(df)))
    ax1.set_xticklabels([t[:15] + "..." for t in df['task']], rotation=45)
    ax1.set_ylabel('Iterations')
    ax1.set_title('Iterations Required by Task')
    
    # Success rate
    ax2 = axes[0, 1]
    success_rate = df['success'].sum() / len(df) * 100
    ax2.pie([success_rate, 100-success_rate], 
            labels=['Success', 'Failed'],
            colors=['lightgreen', 'lightcoral'],
            autopct='%1.1f%%')
    ax2.set_title('Overall Success Rate')
    
    # Final scores
    ax3 = axes[1, 0]
    ax3.scatter(df['iterations'], df['final_score'], s=100, alpha=0.6)
    ax3.set_xlabel('Iterations')
    ax3.set_ylabel('Final Score')
    ax3.set_title('Iterations vs Final Score')
    ax3.grid(True, alpha=0.3)
    
    # Average convergence
    ax4 = axes[1, 1]
    avg_iterations = df.groupby('success')['iterations'].mean()
    ax4.bar(['Failed', 'Success'], avg_iterations)
    ax4.set_ylabel('Average Iterations')
    ax4.set_title('Average Iterations by Outcome')
    
    plt.tight_layout()
    plt.show()
    
    return df

# Run analysis
convergence_data = await analyze_convergence_patterns()
print("\nConvergence Analysis Summary:")
print(convergence_data)

## Key Takeaways

1. **Iterative Refinement Works**:
   - Each iteration improves code quality
   - Error feedback drives targeted improvements
   - Success rate increases with iterations

2. **Agent Architecture Benefits**:
   - State tracking enables learning from history
   - Multiple strategies prevent getting stuck
   - Confidence tracking helps termination decisions

3. **Feedback Loop Importance**:
   - Specific error analysis → Better fixes
   - Pattern recognition → Faster convergence
   - Memory of successes → Reusable solutions

4. **Practical Implementation Insights**:
   - Start simple, refine incrementally
   - Track metrics for improvement visibility
   - Balance iteration count vs quality
   - Use domain knowledge in feedback

5. **Extensions and Applications**:
   - Can adapt to any code generation domain
   - Feedback mechanism is customizable
   - Agent learns task-specific patterns
   - Enables automated code improvement